In [1]:
!pip install -r requirements.txt

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
!pip install -e .

Obtaining file:///tf/notebooks/EPT/1_taxonomist
  Attempting uninstall: taxonomist
    Found existing installation: taxonomist 0.0.3
    Can't uninstall 'taxonomist'. No files were found to uninstall.
  Running setup.py develop for taxonomist
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
df = pd.read_csv('data/processed/EPT-14/01_EPT_processed_5splits_Species_DNA.csv')
df.head()

,Specimen ID,Sample Name/Number,Species Name,Image File Name,Other Notes,Max Feret Diameter,Perimeter,Area,Holes,Area+Holes,...,Genus_DNA,Species_DNA,Label_DNA,read count_DNA,individual,0,1,2,3,4
0,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,1_4_F12_1_2023_12_14-09-51-53-989.PNG,C25_T_mit,352,923,25295,19,25314,...,Goera,Goera pilosa,Goera pilosa,215998.0,4_F12,train,train,val,train,test
1,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,2_4_F12_1_2023_12_14-09-51-54-000.PNG,C25_T_mit,367,1034,28245,168,28413,...,Goera,Goera pilosa,Goera pilosa,215998.0,4_F12,train,train,val,train,test
2,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,1_4_F12_1_2023_12_14-09-51-54-013.PNG,C25_T_mit,400,1093,28301,21,28322,...,Goera,Goera pilosa,Goera pilosa,215998.0,4_F12,train,train,val,train,test
3,4_F12,4_F12_1,PMR_ExStream22_Main_EPT,2_4_F12_1_2023_12_14-09-51-54-025.PNG,C25_T_mit,380,999,29520,43,29563,...,Goera,Goera pilosa,Goera pilosa,215998.0,4_F12,train,train,val,train,test
4,4_F12,4_F12_2,PMR_ExStream22_Main_EPT,1_4_F12_2_2023_12_14-09-52-19-798.PNG,C25_T_mit,239,645,16395,1,16396,...,Goera,Goera pilosa,Goera pilosa,215998.0,4_F12,train,train,val,train,test


### Training pre-trained CNNs with frozen weights

In [7]:
import os
import glob

def list_files_with_prefix(directory, prefix, model):
    # Define the pattern for the files to search
    pattern = os.path.join(directory, f"{prefix}*")
    
    # Use glob to find files matching the pattern
    files = glob.glob(pattern)
    
    return files

def list_ckpts(fold_directory):
    # Define the pattern for the files to search    
    pattern = os.path.join(fold_directory, '*.ckpt')
    
    # Use glob to find files matching the pattern, with recursive search
    ckpt_files = glob.glob(pattern)
    
    filtered_files = [f for f in ckpt_files if model in f if '_241202' in f if not f.endswith('_last.ckpt')]     # Filter out files ending with '_last.ckpt' and only return desired model
    
    # Return the first file from the filtered list or None if the list is empty
    return filtered_files

def extend_fold_dict(directory):
    for subfolder in directory:
        folds = [f.path for f in os.scandir(subfolder) if f.is_dir() and not f.name.startswith('.')]
        # Filter out dotfiles such as '.ipynb_checkpoint' files 
        for fold in folds:
            fold_dict[int(fold[-1])].extend(list_ckpts(fold))      

In [8]:
# Searching for the best model checkpoint in the directory
directory = 'outputs/biodiscover/'
prefix = 'fV1-EPT-14'
model = 'efficientnet_b0'

    
fold_dict={}
for number in range(0,5):
    fold_dict[number] = []

# List the files
setting_folders = list_files_with_prefix(directory, prefix, model)

extend_fold_dict(setting_folders)

In [9]:
fold_dict

{0: ['outputs/biodiscover/fV1-EPT-14-Family_DNA_efficientnet_b0/f0/fV1-EPT-14-Family_DNA_efficientnet_b0_f0_241202-0953-608e_epoch05_val-loss0.45.ckpt',
  'outputs/biodiscover/fV1-EPT-14-Genus_DNA_efficientnet_b0/f0/fV1-EPT-14-Genus_DNA_efficientnet_b0_f0_241202-1006-96fd_epoch05_val-loss0.47.ckpt',
  'outputs/biodiscover/fV1-EPT-14-Order_DNA_efficientnet_b0/f0/fV1-EPT-14-Order_DNA_efficientnet_b0_f0_241202-1020-7554_epoch03_val-loss0.48.ckpt',
  'outputs/biodiscover/fV1-EPT-14-Species_DNA_efficientnet_b0/f0/fV1-EPT-14-Species_DNA_efficientnet_b0_f0_241202-1033-49e6_epoch00_val-loss0.45.ckpt'],
 1: ['outputs/biodiscover/fV1-EPT-14-Family_DNA_efficientnet_b0/f1/fV1-EPT-14-Family_DNA_efficientnet_b0_f1_241202-1047-5446_epoch03_val-loss0.49.ckpt',
  'outputs/biodiscover/fV1-EPT-14-Genus_DNA_efficientnet_b0/f1/fV1-EPT-14-Genus_DNA_efficientnet_b0_f1_241202-1100-da43_epoch02_val-loss0.47.ckpt',
  'outputs/biodiscover/fV1-EPT-14-Order_DNA_efficientnet_b0/f1/fV1-EPT-14-Order_DNA_efficientnet_

In [ ]:
import wandb
wandb.login() #581221dff0063a897a4eff9d285fa871a2c8e7f2

In [ ]:
# 01_ept-14-biomass was used to determine the learning rate for 'Log_weights'
# 02_ept-14-biomass was used to train for 100 epochs on 'Log_weights'
# 03_ept-14-biomass was used to pre-train for 100 epochs on 'Species_DNA', 'Genus_DNA', 'Familiy_DNA', 'Order_DNA'

# f'02_EPT-14-biomass-{label}-{model}' was used to pre-train for 100 epochs on 'Species_DNA', 'Genus_DNA', 'Familiy_DNA', 'Order_DNA' wit early stopping = True and 20 epochs patience

# Train biomass estimation on all 5 folds of previously created splits.
for fold, ls in fold_dict.items():
    for file in ls:
        if 'Species_DNA' in file:
            label = 'Species_DNA' # only used for naming, the train label is log_weights
        if 'Genus_DNA' in file:
            label = 'Genus_DNA' # only used for naming, the train label is log_weights
        if 'Family_DNA' in file:
            label = 'Family_DNA' # only used for naming, the train label is log_weights
        if 'Order_DNA' in file:
            label = 'Order_DNA' # only used for naming, the train label is log_weights
        !python scripts/02_train.py \
            --data_folder "../10_images/" \
            --dataset_config "conf/user_datasets.py" \
            --dataset_name "biodiscover" \
            --csv_path "data/processed/EPT-14/01_EPT_processed_5splits_Species_DNA.csv" \
            --ckpt_path {file} \
            --label "Log_weights" \
            --fold {fold} \
            --class_map 'none' \
            --imsize 224 \
            --batch_size 256 \
            --aug 'flips-rotate-keep-aspect' \
            --load_to_memory 'False' \
            --model 'efficientnet_b0' \
            --freeze_base 'True' \
            --pretrained 'True' \
            --opt 'adamw' \
            --max_epochs 10 \
            --min_epochs 0 \
            --early_stopping 'False' \
            --early_stopping_patience 10 \
            --criterion 'l1' \
            --lr 0.0001 \
            --auto_lr 'True' \
            --log_dir 'V1-EPT-14' \
            --out_folder 'outputs' \
            --out_prefix f'V1-EPT-14-{label}' \
            --deterministic 'True' \
            --precision '16-mixed' \
            --resume 'False'

In [ ]:
import os
import glob

def list_files_with_prefix(directory, prefix):
    # Define the pattern for the files to search
    pattern = os.path.join(directory, f"{prefix}*")
    
    # Use glob to find files matching the pattern
    files = glob.glob(pattern)
    
    return files

def list_ckpts(fold_directory):
    # Define the pattern for the files to search    
    pattern = os.path.join(fold_directory, '*.ckpt')
    
    # Use glob to find files matching the pattern, with recursive search
    ckpt_files = glob.glob(pattern)
    
    filtered_files = [f for f in ckpt_files if '_241202' in f and not f.endswith('_last.ckpt')]
     # Filter out files ending with '_last.ckpt' and the classification models (pre-training)
    
    # Return the first file from the filtered list or None if the list is empty
    return filtered_files

def extend_fold_dict(directory):
    for subfolder in directory:
        folds = [f.path for f in os.scandir(subfolder) if f.is_dir()]
        for fold in folds:
            fold_dict[int(fold[-1])].extend(list_ckpts(fold))
        
directory = 'outputs/biodiscover/'
prefix = 'fV1-EPT-14'

    
fold_dict={}
for number in range(0,5):
    fold_dict[number] = []

# List the files in fold_dict
setting_folders = list_files_with_prefix(directory, prefix)

extend_fold_dict(setting_folders)

In [ ]:
fold_dict

In [ ]:
for fold, ls in fold_dict.items():
    for file in ls:
        # Automatic predictiongs using the first (best) model checkpoint in the directory.
        !python scripts/03_predict.py \
            --data_folder "../10_images/" \
            --dataset_config "conf/user_datasets.py" \
            --dataset_name "biodiscover" \
            --csv_path "data/processed/EPT-14/01_EPT_processed_5splits_Species_DNA.csv" \
            --label "Log_weights" \
            --fold {fold} \
            --class_map 'none' \
            --imsize 224 \
            --batch_size 512 \
            --aug 'none' \
            --out_folder 'outputs' \
            --tta 'False' \
            --out_prefix 'fV1-EPT-14' \
            --ckpt_path {file}

In [ ]:
import os
import glob


def list_files_with_prefix(directory, prefix):
    pattern = os.path.join(directory, f"{prefix}*")
    files = glob.glob(pattern)
    
    return files

def list_csvs(directory): 
    pattern = os.path.join(directory, 'predictions', 'biodiscover_none', '*.csv')
    csv_files = glob.glob(pattern)
    csv_files = [f for f in csv_files if '_241202' in f]
    return csv_files

def extend_run_dict_csv(directory):
    folds = [f.path for f in os.scandir(directory) if f.is_dir()]
    for fold in folds:
        run_dict_csv[directory].extend(list_csvs(fold))
        
directory = 'outputs/biodiscover/'
prefix = 'fV1-EPT-14'        
        
run_dict_csv={}
for run in list_files_with_prefix(directory, prefix):
    run_dict_csv[run] = []
    
setting_folders = list_files_with_prefix(directory, prefix)

for setting in setting_folders:
    extend_run_dict_csv(setting)

In [ ]:
run_dict_csv

In [ ]:
import pandas as pd
from pathlib import Path

ref = pd.read_csv("data/processed/EPT-14/01_EPT_processed_5splits_Species_DNA.csv")

for run, fold_ls in run_dict_csv.items():
    print('joining '+ run)
    df_f0 = pd.read_csv(fold_ls[0])
    df_f1 = pd.read_csv(fold_ls[1])
    df_f2 = pd.read_csv(fold_ls[2])
    df_f3 = pd.read_csv(fold_ls[3])
    df_f4 = pd.read_csv(fold_ls[4])
    df = pd.concat([df_f0, df_f1, df_f2, df_f3, df_f4])

    if 'Species_DNA' in run:
        label = 'Species_DNA' # only used for naming, the train label is log_weights
    if 'Genus_DNA' in run:
        label = 'Genus_DNA' # only used for naming, the train label is log_weights
    if 'Family_DNA' in run:
        label = 'Family_DNA' # only used for naming, the train label is log_weights
    if 'Order_DNA' in run:
        label = 'Order_DNA' # only used for naming, the train label is log_weights
        
    # Get just the filename from the full path
    df["fname"] = df["fname"].apply(lambda x: Path(x).name)
    preds = df[["fname", "y_pred"]].set_index("fname")

    # Join the dataframes. 'inner' to keep only the rows that are in both dataframes
    results_df = ref.join(preds, how="inner", on="Image File Name")  

    # Choose wanted columns
    results_df = results_df[['Log_weights', "y_pred", "individual"]]

    # Grouped by individual
    results_df_grouped = results_df.groupby(["individual"]).agg(lambda x: pd.Series.mode(x)[0])

    results_df_grouped.to_csv(f'{run}/fV1-EPT-14_{label}_biomass.csv')    

### Training pre-trained CNNs with unfrozen weights

In [ ]:
import os
import glob

def list_files_with_prefix(directory, prefix, model):
    # Define the pattern for the files to search
    pattern = os.path.join(directory, f"{prefix}*")
    
    # Use glob to find files matching the pattern
    files = glob.glob(pattern)
    
    return files

def list_ckpts(fold_directory):
    # Define the pattern for the files to search    
    pattern = os.path.join(fold_directory, '*.ckpt')
    
    # Use glob to find files matching the pattern, with recursive search
    ckpt_files = glob.glob(pattern)
    
    filtered_files = [f for f in ckpt_files if '_241202' in f and not f.endswith('_last.ckpt')]
    # Filter out files ending with '_last.ckpt' and the classification models (pre-training)
    
    # Return the first file from the filtered list or None if the list is empty
    return filtered_files

def extend_fold_dict(directory):
    for subfolder in directory:
        folds = [f.path for f in os.scandir(subfolder) if f.is_dir() and not f.name.startswith('.')]
        # Filter out dotfiles such as '.ipynb_checkpoint' files 
        for fold in folds:
            fold_dict[int(fold[-1])].extend(list_ckpts(fold))  

In [ ]:
# Searching for the best model checkpoint in the directory
directory = 'outputs/biodiscover/'
prefix = 'fV1-EPT-14'
model = 'efficientnet_b0'

    
fold_dict={}
for number in range(0,5):
    fold_dict[number] = []

# List the files
setting_folders = list_files_with_prefix(directory, prefix, model)

extend_fold_dict(setting_folders)

In [ ]:
fold_dict

In [ ]:
import wandb
wandb.login() #581221dff0063a897a4eff9d285fa871a2c8e7f2

In [ ]:
# 01_ept-14-biomass was used to determine the learning rate for 'Log_weights'
# 02_ept-14-biomass was used to train for 100 epochs on 'Log_weights'
# 03_ept-14-biomass was used to pre-train for 100 epochs on 'Species_DNA', 'Genus_DNA', 'Familiy_DNA', 'Order_DNA'

# f'02_EPT-14-biomass-{label}-{model}' was used to pre-train for 100 epochs on 'Species_DNA', 'Genus_DNA', 'Familiy_DNA', 'Order_DNA' wit early stopping = True and 20 epochs patience

# Train biomass estimation on all 5 folds of previously created splits.
for fold, ls in fold_dict.items():
    for file in ls:
        if 'Species_DNA' in file:
            label = 'Species_DNA' # only used for naming, the train label is log_weights
        if 'Genus_DNA' in file:
            label = 'Genus_DNA' # only used for naming, the train label is log_weights
        if 'Family_DNA' in file:
            label = 'Family_DNA' # only used for naming, the train label is log_weights
        if 'Order_DNA' in file:
            label = 'Order_DNA' # only used for naming, the train label is log_weights
        !python scripts/02_train.py \
            --data_folder "../10_images/" \
            --dataset_config "conf/user_datasets.py" \
            --dataset_name "biodiscover" \
            --csv_path "data/processed/EPT-14/01_EPT_processed_5splits_Species_DNA.csv" \
            --ckpt_path {file} \
            --label "Log_weights" \
            --fold {fold} \
            --class_map 'none' \
            --imsize 224 \
            --batch_size 256 \
            --aug 'flips-rotate-keep-aspect' \
            --load_to_memory 'False' \
            --model 'efficientnet_b0' \
            --freeze_base 'False' \
            --pretrained 'True' \
            --opt 'adamw' \
            --max_epochs 10 \
            --min_epochs 0 \
            --early_stopping 'False' \
            --early_stopping_patience 10 \
            --criterion 'l1' \
            --lr 0.0001 \
            --auto_lr 'True' \
            --log_dir 'V1-EPT-14-unfrozen' \
            --out_folder 'outputs' \
            --out_prefix f'V1-EPT-14-{label}-unfrozen' \
            --deterministic 'True' \
            --precision '16-mixed' \
            --resume 'False'

In [ ]:
import os
import glob

def list_files_with_prefix(directory, prefix):
    # Define the pattern for the files to search
    pattern = os.path.join(directory, f"{prefix}*")
    
    # Use glob to find files matching the pattern
    files = glob.glob(pattern)
    
    return files

def list_ckpts(fold_directory):
    # Define the pattern for the files to search    
    pattern = os.path.join(fold_directory, '*.ckpt')
    
    # Use glob to find files matching the pattern, with recursive search
    ckpt_files = glob.glob(pattern)
    
    filtered_files = [f for f in ckpt_files if 'unfrozen' in f and not f.endswith('_last.ckpt')]
     # Filter out files ending with '_last.ckpt' and the classification models (pre-training)
    
    # Return the first file from the filtered list or None if the list is empty
    return filtered_files

def extend_fold_dict(directory):
    for subfolder in directory:
        folds = [f.path for f in os.scandir(subfolder) if f.is_dir() and not f.name.startswith('.')]
        for fold in folds:
            fold_dict[int(fold[-1])].extend(list_ckpts(fold))
        
directory = 'outputs/biodiscover/'
prefix = 'fV1-EPT-14'

    
fold_dict={}
for number in range(0,5):
    fold_dict[number] = []

# List the files in fold_dict
setting_folders = list_files_with_prefix(directory, prefix)

extend_fold_dict(setting_folders)

In [ ]:
fold_dict

In [ ]:
for fold, ls in fold_dict.items():
    for file in ls:
        # Automatic predictiongs using the first (best) model checkpoint in the directory.
        !python scripts/03_predict.py \
            --data_folder "../10_images/" \
            --dataset_config "conf/user_datasets.py" \
            --dataset_name "biodiscover" \
            --csv_path "data/processed/EPT-14/01_EPT_processed_5splits_Species_DNA.csv" \
            --label "Log_weights" \
            --fold {fold} \
            --class_map 'none' \
            --imsize 224 \
            --batch_size 512 \
            --aug 'none' \
            --out_folder 'outputs' \
            --tta 'False' \
            --out_prefix 'fV1-EPT-14' \
            --ckpt_path {file}

In [ ]:
import os
import glob


def list_files_with_prefix(directory, prefix):
    pattern = os.path.join(directory, f"{prefix}*")
    files = glob.glob(pattern)
    
    return files

def list_csvs(directory): 
    pattern = os.path.join(directory, 'predictions', 'biodiscover_none', '*.csv')
    csv_files = glob.glob(pattern)
    csv_files = [f for f in csv_files if 'unfrozen' in f]
    return csv_files

def extend_run_dict_csv(directory):
    folds = [f.path for f in os.scandir(directory) if f.is_dir()]
    for fold in folds:
        run_dict_csv[directory].extend(list_csvs(fold))
        
directory = 'outputs/biodiscover/'
prefix = 'fV1-EPT-14'        
        
run_dict_csv={}
for run in list_files_with_prefix(directory, prefix):
    run_dict_csv[run] = []
    
setting_folders = list_files_with_prefix(directory, prefix)

for setting in setting_folders:
    extend_run_dict_csv(setting)
    
run_dict_csv = {k: v for k, v in run_dict_csv.items() if 'unfrozen' in k}

In [ ]:
run_dict_csv

In [ ]:
import pandas as pd
from pathlib import Path

ref = pd.read_csv("data/processed/EPT-14/01_EPT_processed_5splits_Species_DNA.csv")

for run, fold_ls in run_dict_csv.items():
    print('joining '+ run)
    df_f0 = pd.read_csv(fold_ls[0])
    df_f1 = pd.read_csv(fold_ls[1])
    df_f2 = pd.read_csv(fold_ls[2])
    df_f3 = pd.read_csv(fold_ls[3])
    df_f4 = pd.read_csv(fold_ls[4])
    df = pd.concat([df_f0, df_f1, df_f2, df_f3, df_f4])

    if 'Species_DNA' in run:
        label = 'Species_DNA' # only used for naming, the train label is log_weights
    if 'Genus_DNA' in run:
        label = 'Genus_DNA' # only used for naming, the train label is log_weights
    if 'Family_DNA' in run:
        label = 'Family_DNA' # only used for naming, the train label is log_weights
    if 'Order_DNA' in run:
        label = 'Order_DNA' # only used for naming, the train label is log_weights
        
    # Get just the filename from the full path
    df["fname"] = df["fname"].apply(lambda x: Path(x).name)
    preds = df[["fname", "y_pred"]].set_index("fname")

    # Join the dataframes. 'inner' to keep only the rows that are in both dataframes
    results_df = ref.join(preds, how="inner", on="Image File Name")  

    # Choose wanted columns
    results_df = results_df[['Log_weights', "y_pred", "individual"]]

    # Grouped by individual
    results_df_grouped = results_df.groupby(["individual"]).apply(lambda group: group[(group >= group.quantile(0.05)) & (group <= group.quantile(0.95))].mean())

    results_df_grouped.to_csv(f'{run}/fV1-EPT-14_{label}-unfrozen_biomass.csv')    

In [ ]:
## Until here, the code works fine. The following parts are still more or less experimental and for sure not cleaned up.

In [ ]:
import pandas as pd
confusion_df = pd.read_csv('outputs/biodiscover/02_ept-14-biomass_efficientnet_b0/01_EPT_processed_5splits_Species_DNA_grouped_predictions.csv')

confusion_matrixX(confusion_df['y_true'], confusion_df['y_pred'], 'union')